In [1]:
import os
import pandas as pd
import plotly.express as px
import progressbar
import json
import numpy as np
import nltk
from nltk.corpus import stopwords
import string
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize
pd.set_option('display.max_colwidth', None)

In [2]:
def remove_stopwords(token_vector, stopwords_list):
    return token_vector.apply(lambda token_list: [word for word in token_list if word not in stopwords_list])

In [3]:
def remove_punctuation(token_vector):
    return token_vector.apply(lambda token_list: [word for word in token_list if word not in string.punctuation])

In [4]:
def jaccard_similarity_score(original, translation):
    intersect = set(original).intersection(set(translation))
    union = set(original).union(set(translation))
    try:
        return len(intersect) / len(union)
    except ZeroDivisionError:
        return 0

## Get the paths and build table names for iteration

In [5]:
data_path = '../src/data'
mapping_corpus_path = data_path + r'/product/lspc2020_to_tablecorpus'
mapping_corpus_path_2 = data_path + r'/product/lspc2020_to_tablecorpus/Cleaned'
table_corpus_path = data_path + r'/product/product_top100/cleaned'
table_corpus_path_with_id = data_path + r'/product/product_top100/cleaned/with_id'
table_corpus_path2 = data_path + r'/product/product_minimum3/cleaned/with_id'

In [6]:
zip_files_mapping = [file for file in os.listdir(mapping_corpus_path_2) if file.endswith('.json.gz')]
zip_files_tables = [file for file in os.listdir(table_corpus_path) if file.endswith('.json.gz')]

## Match the number dictionaries with the information about the brand


In [7]:
df_large = pd.read_json(os.path.join(mapping_corpus_path_2, 'df_large_matched.json'), compression='gzip', orient='records', lines=True)

In [8]:
df_large

,cluster_id,url,row_id,table_id,Valid,name,description,brand
0,59178314,https://www.cultureindoor.com/796-irrigation-arrosage-tuyau-pe-20mm-semi-rigide-rouleau-100m.html,1561,Product_cultureindoor.com_September2020.json.gz,1,"irrigation ,arrosage Tuyau PE 20mm semi rigide rouleau 100m","Tuyau PE 20mm semi rigide rouleau 100m, idéal pour les irrigations de jardins et arrosage automatique, retrouvez toute l'irrigation chez culture indoor le leader du jardinage et de l'hydroponie en france et en europe",FRANCE ARROSAGE
1,70692685,https://www.culturekings.com/products/new-era-new-era-new-orleans-pelicans-nba-940-a-frame-snapback-black,3893,Product_culturekings.com_September2020.json.gz,1,New Era New Orleans Pelicans NBA 9FORTY A-Frame Snapback Black,New Era is back at it again this season with another unique instalment of the 940 silhouette. This time around Culture Kings has the Worldwide Exclusive on these so you can be assured that you won't find them anywhere else. Culture Kings have worked closely with New Era to design a hat that features a slightly higher crown for increased comfort and added style points. The highest quality cotton twill fabric has been used for extreme comfort and durability and the shape is so unique that you just won't find this on any other hat. - Curved Brim - Extra room at the crown- Embroidered team logo- Rear snapback closure,None
2,395606,https://www.culturekings.com/products/new-era-x-nrl-west-tigers-aframe-charcoal-orange,3894,Product_culturekings.com_September2020.json.gz,1,New Era X NRL West Tigers Aframe Charcoal/orange,"The Iconic New Era is the premium siloute of headwear worldwide. The official Major league baseball on field hat, as well as now NRL. It's fit is can be worn with flat visor or curved, The best quality hat in the world.",None
3,3361797,https://www.customizedgirl.com/design/2605608/The+Real+Baseball+Moms,9318,Product_customizedgirl.com_September2020.json.gz,1,The Real Baseball Moms (2XL),"Are you a real baseball mom fan? Customize a cute design spoof on the \""real housewives\"" show. Add your team's name and your athlete's name.",CustomizedGirl.com
4,235755,https://www.customizedgirl.com/design/2613423/Mentally+Dating+Ronaldo,9319,Product_customizedgirl.com_September2020.json.gz,1,Mentally Dating Ronaldo (2XL),"Mentally dating Christiano Ronaldo. Yeah girl, me too. He literally has changed the game of soccer into something women watch just to see him. That hair, those abs, the smile. It's too much to handle. If I was the goalie, I'd let him score every single time.",CustomizedGirl.com
...,...,...,...,...,...,...,...,...
24688354,8653717,https://shop-list.com/women/utuwa-tatara/sg3232-03,1620,Product_shop-list.com_September2020.json.gz,1,vZXEC h[ yAv[gZbg(Vf) v[g Mtg ,yPrincessRoyalDreams-vZXCh[-zyAv[gZbg(Vf)(v[g2_̃ZbgłB)fBYj[EvZX݂hCEFfBOhC[WBFȂXg[[ƋɁAK𑡂܂B܂AϔȂ̂ŁAMtgƂĂ߂łBFB₲Ƒւ̃v[gMtgɂł傤B+++V[Y֘Ai+++EyAOXZbg(Vf)EyAXeOXZbg(Vf)EXeOX&amp;v[gZbg(AG)EXeOX&amp;v[gZbg(vcF)TCY21~H2.3cmeʁ[ގ({)@v[g͉_gpĂ܂BӂƓdqWOKHNGNG̑t̎戵mFB݌ɂ̏i͓XʓX܂ł̔Ă܂B݌ɐ̍XV͐sĂ܂A󋵂ɂĂ͏ii؂ƂȂ邱Ƃ܂B܂i񂹂ɂ肨͂܂łɂԂꍇ܂Bqlɂ͕KA܂Aꃁ[J[ŕi؂ꓙ̗Rɂח肪ȂAׂ͓xꍇ̓LZĂꍇ܂Ƃ炩߂B,None
24688355,22258637,https://shop-list.com/women/youmotto/famille-ltm,1621,Product_shop-list.com_September2020.json.gz,1,t@~[ OgC}bg80~60cm[gC }bg gC^[ Jg[ _ i` IV AC{[uE n O CeA 􂦂ېOK tH ~ `RȊO],yiڍׁzyގzpC/|GXe100yTCYz80~60cmyYzyɂĂ̂ӁzEŏ̐͗VітAڐ肷邱Ƃ܂̂ŁAK̕ƕĐĂB͎኱kނƂ܂Agpɂ͂܂BEh@iȂȂ߃h܂ށj͐΂ɂgpɂȂȂłBpCiёj̔`̘č݂ƂȂ܂̂Ŏ􂢂ߒv܂ByӁzE܂肽ŕۊǂĂׁAiɐ܂ȂtĂ邱Ƃ܂Bx󂵂Ă܂ƁA܂ȂڗȂȂ܂BEg̃j^[̐ݒɂAF̌قȂ邱Ƃ܂BB,None
24688356,41434658,https://shop.agwaycapecod.com/products/019014710959/adult-small-breed-dog-food-5-lb,126,Product_agwaycapecod.com_September2020.json.gz,1,"Adult Small Breed Dog Food, 5-Lb.","Eukanuba, 5 LB, Adult Small Breed Dry Dog Food, Has A Higher Amount Of Fat To Meet The Unique Energy Levels, Kibble Size Suits Their Metabolisms & Stomach Size, Promotes Active, Daily Living With Antioxidants For Immune Health, Fiber For Healthy Digestion, Fatty Acids For Coat & Skin Health & Other Natural Nutrients, Natural F

In [ ]:
df_joined_clothes = df_large.merge(df_clothes_cleaned, left_on=['table_id','row_id'], right_on = ['table_id','row_id'], how='left')

In [ ]:
df_joined_electronics.to_json(mapping_corpus_path_2 + '/joined_electronics_v2.json', compression='gzip', orient='records', lines=True)

In [6]:
df_joined_clothes.to_json(mapping_corpus_path_2 + '/joined_clothes_v2.json', compression='gzip', orient='records', lines=True)

In [6]:
df_joined_electronics = pd.read_json(os.path.join(mapping_corpus_path_2, 'joined_electronics_v2.json'), compression='gzip', orient='records', lines=True)

In [7]:
df_joined_clothes = pd.read_json(os.path.join(mapping_corpus_path_2, 'joined_clothes_v2.json'), compression='gzip', orient='records', lines=True)

In [22]:
df_joined_clothes = pd.read_csv(os.path.join(mapping_corpus_path_2, 'clothes_clusters_all_8_tables_post_processed.csv'))

In [8]:
df_joined_electronics = pd.read_csv(os.path.join(mapping_corpus_path_2, 'electronics_clusters_all_10_tables_post_processed.csv'))

In [23]:
df_joined_clothes

,Unnamed: 0,Unnamed: 0.1,cluster_id,url,row_id,table_id,Valid,name,description,brand_x,brand_y,tokens
0,4,15925,78110534,https://www.cn.forzieri.com/chn/product_view.asp?l=chn&c=chn&dept_id=18&sku=dg130320-027-00,3400,Product_forzieri.com_September2020.json.gz,1,"small \""devotion\"" bag","Leather handle, detachable shoulder strap, front plate with logo jewel heart, magnetic closure, inner flat pocket, leather inner, gold finish metal accessories, made in italy.Sensorial gift wrap available",Dolce & Gabbana żΰ,NaN,"['small', ""''"", 'devotion\\', ""''"", 'bag']"
1,5,18456,79412,https://www.lookingglassphoto.com/peak-design-the-everyday-messenger-charcoal-15.html,2600,Product_lookingglassphoto.com_September2020.json.gz,1,peak design peak design the everyday messenger- charcoal 15”,NaN,PEAK DESIGN,NaN,"['peak', 'design', 'peak', 'design', 'everyday', 'messenger-', 'charcoal', '15', '”']"
2,6,19121,587935,https://soloptical.net/es/rayban-reg-justin-color-mix-rb4165-622-55.html,558,Product_soloptical.net_September2020.json.gz,1,rayban ® justin color mix rb4165 622/55,Gafas de sol de última colección y diseños novedosos que representan uno de los principales accesorios de la moda actual,NaN,NaN,"['rayban', '®', 'justin', 'color', 'mix', 'rb4165', '622/55']"
3,7,19932,391605,https://www.whisknyc.com/cake-stand-w-glass-dome-anchor.html,3052,Product_whisknyc.com_September2020.json.gz,1,"13\"" glass cake stand with dome",NaN,NaN,NaN,"['13\\', ""''"", 'glass', 'cake', 'stand', 'dome']"
4,8,27046,2434654,https://avcstore.com/products/saramonic-sr-tm1-cardioid-condenser-shotgun-microphone,803,Product_avcstore.com_September2020.json.gz,1,saramonic sr-tm1 cardioid condenser shotgun microphone,"For TV, Broadcast & Video ProductionCardioid Polar PatternEnhanced Rear Signal Rejection40 Hz to 20 kHz Frequency ResponseInternal 150-Hour Lithium-Ion BatterySupports 48V Phantom PowerSelectable Pad, Roll-Off & High BoostXLR 3-Pin OutputIncludes Windscreen & Mic ClipIncludes XLR Cable & USB Charging CableThe Saramonic SR-TM1 is a medium-length cardioid condenser shotgun microphone offering high SPL handling and full-bandwidth frequency reproduction for sound recordists, boom operators, and videographers in broadcast, video, and television production. The cardioid polar pattern ensures reliable directivity with enhanced rejection of signal at the rear of capsule. Operate the SR-TM1 via 48 VDC phantom power or the internal lithium-ion battery, which delivers up to 150 hours of use and recharges through the supplied micro-USB cable.Selection of the -10 dB pad allows the SR-TM1 to capture loud sound sources without inducing distortion while the 40 Hz to 20 kHz frequency response accurately conveys your source's tone. Customize the SR-TM1's sonic performance with the switchable 150 Hz high-pass filter for minimized low-frequency rumble, and the 6 dB high-frequency boost for improved vocal clarity.Connect the SR-TM1 to your mixer or microphone preamp via the XLR 3-pin output. It includes a foam windscreen, short XLR cable, and mic clip with a camera shoe mount.",SARAMONIC,NaN,"['saramonic', 'sr-tm1', 'cardioid', 'condenser', 'shotgun', 'microphone']"
...,...,...,...,...,...,...,...,...,...,...,...,...
6028,9729,24666813,434734,https://www.scottscycle.com/product/chamois-buttr-original-formula-chamois-cream-184928-1.htm,69,Product_scottscycle.com_September2020.json.gz,1,original formula chamois cream,"Chamois Butt’r Original is a non-greasy skin lubricant developed by cyclists for use with all chamois. Chamois Butt’r isn’t just for cyclists! It lubricates, soothes, and softens the skin of all...",NaN,NaN,"['original', 'formula', 'chamois', 'cream']"
6029,9730,24668368,5121322,https://urbanpalms.com/product/lg-bottle-palm-tree-hyophorbe-lagenicaulis,5,Product_urbanpalms.com_September2020.json.gz,1,lg. bottle palm tree (hyophorbe lagenicaulis),Unique and unlike any other palm Bottle Palm is a distinctive addition to any plant collection. Contained in a plante

## Get information about electronic clusters and train model

In [11]:
df_grouped_electronics = df_joined_electronics.groupby('cluster_id').count()
# only look at clusters that have at least one brand associated
df_set_electronics = df_grouped_electronics.reset_index()[['cluster_id','table_id']].rename(columns={'table_id':'Amount'})

In [12]:
df_10_electronics=df_set_electronics[df_set_electronics['Amount']>10]

In [13]:
df_10_electronics

,cluster_id,Amount
2,3668,11
3,6076,14
4,6443,13
5,6505,13
6,6690,17
...,...,...
615,52362093,20
616,53457772,61
617,64651308,11
618,66614988,12


In [10]:
#df_10_electronics.to_csv('electronics10.csv')

In [15]:
# We discard all clusters with less than 2 entries, cause we cannot match anything there, so 1,6 million clusters remain
df_set_electronics=df_set_electronics[df_set_electronics['Amount']>1]
df_15_electronics=df_set_electronics[df_set_electronics['Amount']>12]
df_15_electronics

,cluster_id,Amount
3,6076,14
4,6443,13
5,6505,13
6,6690,17
8,7366,13
...,...,...
611,48051979,13
613,48051984,13
615,52362093,20
616,53457772,61


In [ ]:
#merge brand name to cluster amount
df_cluster_brand = df_15_electronics[df_15_electronics['Amount']<200].merge(df_joined_electronics.dropna()[['cluster_id','brand_y']].drop_duplicates('cluster_id', keep='last'), left_on=['cluster_id'], right_on = ['cluster_id'], how='left')
df_cluster_brand

In [16]:
#clean product column and lowercase
df_joined_electronics=df_joined_electronics.dropna(subset = ['name'])
df_joined_electronics['name'] = df_joined_electronics['name'].apply(lambda row: row.lower())
df_joined_electronics
#get only cluster ids with at least one brand electronics
df_compare_electronics = df_joined_electronics[df_joined_electronics['cluster_id'].isin(df_set_electronics['cluster_id'].tolist())]
#merge with set to get amount of tables per cluster in overview
df_compare_electronics = df_compare_electronics.merge(df_set_electronics, left_on=['cluster_id'], right_on = ['cluster_id'], how='left')

In [17]:
#use tokenizer for product names to get tokes for training the model
df_compare_electronics['product_tokes'] = df_compare_electronics['name'].apply(lambda row: word_tokenize(row))
df_compare_electronics['product_tokes'] = remove_stopwords(df_compare_electronics['product_tokes'],stopwords.words())
df_compare_electronics['product_tokes'] = remove_punctuation (df_compare_electronics['product_tokes'])
#get tagged words
tagged_data = [TaggedDocument(words=_d, tags=[str(i)]) for i, _d in enumerate(df_compare_electronics['product_tokes'])]
# build model and vocabulary
model = Doc2Vec(vector_size=50, min_count = 5, epochs = 25, dm = 0)
model.build_vocab(tagged_data)
# Train model
model.train(tagged_data, total_examples=model.corpus_count, epochs=25)

In [26]:
#get cluster ids for basline products and with that indices of top products to use model
#1524820,47566,6076,14418,28307,33570,39040,51314,99153,215254,685416, 984421 , 1808651,2887810,34506065,47841827,620473,56116,94055, 150211,182246, 516888, 562955 
top_clusters_list = df_15_electronics['cluster_id'].tolist()
index_top_clusters_list=[]
for id in top_clusters_list:
    index_top_clusters_list.append(df_compare_electronics[df_compare_electronics['cluster_id']==id].index[0])

In [28]:
# get most similar products for each of the base clusters and save them if they have more than 5 tables
electronics_clusters_search=[]
for i in index_top_clusters_list:
    similar_doc = model.docvecs.most_similar(f'{i}', topn = 20)
    electronics_clusters_search.append(int(i))
    for index, similarity in similar_doc:
        if df_compare_electronics.iloc[int(index)]['Amount']>8:
            electronics_clusters_search.append(int(index))
    jaccard_score = df_compare_electronics['product_tokes'].apply(lambda row: jaccard_similarity_score(row,df_compare_electronics.iloc[int(i)]['product_tokes']) )
    indizes=sorted(range(len(jaccard_score)), key=lambda i: jaccard_score[i])[-20:]
    for index in indizes:
         if df_compare_electronics.iloc[int(index)]['Amount']>8:
            electronics_clusters_search.append(int(index))    
df_electroncis_final = df_compare_electronics.iloc[electronics_clusters_search]

In [29]:
df_electroncis_final.drop_duplicates('cluster_id', keep='first').to_excel("Final_Electronics_v4.xlsx")

In [ ]:
df_electroncis_final = df_electroncis_final.drop_duplicates('cluster_id', keep='first')

In [ ]:
df_joined_electronics.loc[df_joined_electronics['cluster_id'].isin(df_electroncis_final['cluster_id'].tolist())]

## Cluster statistics for product category clothes

In [24]:
df_grouped_clothes = df_joined_clothes.groupby('cluster_id').count()

In [25]:
# only look at clusters that have at least one brand associated
df_set_clothes = df_grouped_clothes[df_grouped_clothes['brand_y']>0].reset_index()[['cluster_id','table_id']].rename(columns={'table_id':'Amount'})

In [26]:
# We discard all clusters with less than 2 entries, cause we cannot match anything there, so 1,6 million clusters remain
df_set_clothes=df_set_clothes[df_set_clothes['Amount']>1]
df_set_clothes

,cluster_id,Amount
0,16573,6
1,18496,8
2,37366,3
3,37525,9
4,42322,8
...,...,...
282,77602818,15
283,78110534,12
284,78499693,15
285,78736275,8


In [27]:
df_8_clothes=df_set_clothes[df_set_clothes['Amount']>8]
df_8_clothes

,cluster_id,Amount
3,37525,9
5,58043,9
6,70109,10
12,127282,9
15,142594,9
...,...,...
281,77104640,9
282,77602818,15
283,78110534,12
284,78499693,15


In [31]:
df_10_clothes=df_set_clothes[df_set_clothes['Amount']>10]
df_10_clothes

,cluster_id,Amount
16,148199,20
17,150322,11
20,186753,12
24,200956,15
34,308891,13
38,363117,13
39,371522,11
40,390077,13
58,570162,11
66,668889,12


In [19]:
#merge brand name to cluster amount
df_cluster_brand_clothes = df_10_clothes[df_10_clothes['Amount']<400].merge(df_joined_clothes.dropna()[['cluster_id','brand_y']].drop_duplicates('cluster_id', keep='last'), left_on=['cluster_id'], right_on = ['cluster_id'], how='left')
df_cluster_brand_clothes

,cluster_id,Amount,brand_y
0,5310,11,armani
1,37366,14,tag heuer
2,37525,12,coach
3,42322,14,tag heuer
4,58043,12,NaN
...,...,...,...
262,68554513,13,NaN
263,77602818,16,valentino
264,78110534,13,dolce & gabbana
265,78499693,16,dolce & gabbana


In [28]:
df_joined_clothes=df_joined_clothes.dropna(subset = ['name'])
#clean product column and lowercase
df_joined_clothes['name'] = df_joined_clothes['name'].apply(lambda row: row.lower())
df_joined_clothes
#get only cluster ids with at least one brand electronics
df_compare_clothes = df_joined_clothes[df_joined_clothes['cluster_id'].isin(df_set_clothes['cluster_id'].tolist())]
#merge with set to get amount of tables per cluster in overview
df_compare_clothes = df_compare_clothes.merge(df_set_clothes, left_on=['cluster_id'], right_on = ['cluster_id'], how='left')

In [29]:
#use tokenizer for product names to get tokes for training the model
df_compare_clothes['product_tokes'] = df_compare_clothes['name'].apply(lambda row: word_tokenize(row))
df_compare_clothes['product_tokes'] = remove_stopwords(df_compare_clothes['product_tokes'],stopwords.words())
df_compare_clothes['product_tokes'] = remove_punctuation (df_compare_clothes['product_tokes'])
#get tagged words
tagged_data = [TaggedDocument(words=_d, tags=[str(i)]) for i, _d in enumerate(df_compare_clothes['product_tokes'])]
# build model and vocabulary
model = Doc2Vec(vector_size=50, min_count = 5, epochs = 25, dm = 0)
model.build_vocab(tagged_data)
# Train model
model.train(tagged_data, total_examples=model.corpus_count, epochs=25)

In [32]:
#get cluster ids and with that indices of top products to use model
#5310, 58043,104343,142594,174327, 186753,421372,677207,834201, 881202,  895708,939889, 1249086,1290229, 1852022,2459966, 2732926 , 22374915, 22374918, 26097914,44159446, 58592784, 78110534,135583,148199, 200956, 950691, 1592417,2464591
top_clusters_list = df_10_clothes['cluster_id'].tolist()
index_top_clusters_list=[]
for id in top_clusters_list:
    index_top_clusters_list.append(df_compare_clothes[df_compare_clothes['cluster_id']==id].index[0])

In [35]:
# get most similar products for each of the base clusters and save them if they have more than 5 tables
clothes_clusters_search=[]
for i in index_top_clusters_list:
    similar_doc = model.docvecs.most_similar(f'{i}', topn = 20)
    clothes_clusters_search.append(int(i))
    for index, similarity in similar_doc:
        if df_compare_clothes.iloc[int(index)]['Amount']>8:
            clothes_clusters_search.append(int(index))
    jaccard_score = df_compare_clothes['product_tokes'].apply(lambda row: jaccard_similarity_score(row,df_compare_clothes.iloc[int(i)]['product_tokes']) )
    indizes=sorted(range(len(jaccard_score)), key=lambda i: jaccard_score[i])[-20:]
    for index in indizes:
         if df_compare_clothes.iloc[int(index)]['Amount']>8:
            clothes_clusters_search.append(int(index))    
df_clothes_final = df_compare_clothes.iloc[clothes_clusters_search]

In [36]:
df_clothes_final.drop_duplicates('cluster_id', keep='first').to_excel("Final_Clothes_v4.xlsx")

In [60]:
df_joined_clothes = pd.read_csv(os.path.join(mapping_corpus_path_2, 'clothes_clusters_all_8_tables_post_processed.csv'))
df_joined_clothes['category']='clothes'
df_joined_clothes = df_joined_clothes.drop(columns = ['brand_y']).rename(columns={'brand_x':'brand'})

In [61]:
df_joined_electronics = pd.read_csv(os.path.join(mapping_corpus_path_2, 'electronics_clusters_all_10_tables_post_processed.csv'))
df_joined_electronics['category']='electronics'

In [62]:
frames = [df_joined_electronics, df_joined_clothes]
df_concat = pd.concat(frames).drop(columns = ['Unnamed: 0','Unnamed: 0.1','Valid'])
df_concat

,cluster_id,url,row_id,table_id,name,description,brand,tokens,category
0,1001446,https://www.bts.com.au/store/p102/Apple_Lightning_Digital_AV_Adapter.html,90,Product_bts.com.au_September2020.json.gz,apple lightning digital av adapter,"Use the Lightning Digital AV Adapter with your iPhone, iPad or iPod with Lightning connector. The Lightning Digital AV Adapter supports mirroring of what is displayed on your device screen — including apps, presentations, websites, slideshows and more — to your HDMI-equipped TV, display, projector or other compatible display in up to 1080p HD.It also outputs video content — movies, TV shows, captured video — to your big screen in up to 1080p HD. Simply attach the Lightning Digital AV Adapter to the Lightning connector on your device and then to your TV or projector via an HDMI cable (sold separately).",NaN,"['apple', 'lightning', 'digital', 'adapter']",electronics
1,2859551,https://www.icentre.com.mt/apple-lightning-to-usb-cable-1m-2,511,Product_icentre.com.mt_September2020.json.gz,apple lightning to usb cable,"This USB 2.0 cable connects your iPhone, iPad, or iPod with Lightning connector to your computer’s USB port for syncing and charging. Or you can connect to the Apple USB Power Adapter for convenient charging from a wall outlet.",NaN,"['apple', 'lightning', 'usb', 'cable']",electronics
2,323989,https://store.macs-4-u.com.au/usb-c-charge-cable-1-m.html,192,Product_macs-4-u.com.au_September2020.json.gz,usb-c charge cable (1 m),USB-C Charge Cable (1 m),NaN,"['usb-c', 'charge', 'cable', '1']",electronics
3,210849,https://www.cingolanibikeshop.com/specialized-sella-power-comp-nero-143.html,447,Product_cingolanibikeshop.com_September2020.json.gz,specialized sella power comp nero 143,SELLA POWER COMP NERO 143,Specialized,"['specialized', 'sella', 'power', 'comp', 'nero', '143']",electronics
4,345616,https://www.scan.co.uk/products/6tb-wd-red-pro-wd6003ffbx-35-nas-hdd-sata-iii-6gb-s-7200rpm-256mb-cache-shock-sensor-ncq-oem,4945,Product_scan.co.uk_September2020.json.gz,"wd red pro 6tb 3.5\"" sata nas hdd/hard drive","6TB WD Red Pro WD6003FFBX, 3.5\"" NAS HDD, SATA III 6Gb/s, 7200rpm, 256MB Cache, Shock Sensor, NCQ, OEM",WD,"['wd', 'red', 'pro', '6tb', '3.5\\', ""''"", 'sata', 'hdd/hard', 'drive']",electronics
...,...,...,...,...,...,...,...,...,...
6028,434734,https://www.scottscycle.com/product/chamois-buttr-original-formula-chamois-cream-184928-1.htm,69,Product_scottscycle.com_September2020.json.gz,original formula chamois cream,"Chamois Butt’r Original is a non-greasy skin lubricant developed by cyclists for use with all chamois. Chamois Butt’r isn’t just for cyclists! It lubricates, soothes, and softens the skin of all...",NaN,"['original', 'formula', 'chamois', 'cream']",clothes
6029,5121322,https://urbanpalms.com/product/lg-bottle-palm-tree-hyophorbe-lagenicaulis,5,Product_urbanpalms.com_September2020.json.gz,lg. bottle palm tree (hyophorbe lagenicaulis),Unique and unlike any other palm Bottle Palm is a distinctive addition to any plant collection. Contained in a planter the Bottle Palm is a shapely centerpiece to any porch patio or well lit indoor area. Native to the Mascarene Islands mature trunks have sometimes attained heights of 20 feet.,Urban Palms,"['lg', 'bottle', 'palm', 'tree', 'hyophorbe', 'lagenicaulis']",clothes
6030,5121322,https://spectrumtelescope.com/product/glass-solar-filter-st1150g,30,Product_spectrumtelescope.com_September2020.json.gz,glass solar filter st1150g,"Your O.D. Measurement:11 1/8” (283mm)to11 3/8” (289mm) Fits:Celestron 10\"" Newt;Orion SkyWatcher 10\""",Spectrum Telescope,"['glass', 'solar', 'filter', 'st1150g']",clothes
6031,1512681,https://www.abiza.co.uk/product/raw-garnet-necklace-january-birthday-gifts-raw-stone-necklace-garnet-jewelry,32,Product_abiza.co.uk_September2020.json.gz,raw garnet necklace,Gorgeous deep red Raw Garnet stones in their most natural state have been hand crafted into pendants and suspending from beautiful shimmering chains. Available

In [63]:
df_grouped= df_concat.groupby('cluster_id').count()
# only look at clusters that have at least one brand associated
df_set = df_grouped.reset_index()[['cluster_id','table_id']].rename(columns={'table_id':'Amount'})

In [64]:
df_10=df_set[df_set['Amount']>10]

In [65]:
df_10

,cluster_id,Amount
2,3668,11
4,6076,14
5,6443,13
6,6505,13
8,6690,17
...,...,...
1069,77602818,15
1070,78110534,12
1072,78499693,15
1074,80097153,59


In [66]:
#clean product column and lowercase
df_concat=df_concat.dropna(subset = ['name'])
df_concat['name'] = df_concat['name'].apply(lambda row: row.lower())
df_concat
#get only cluster ids with at least one brand electronics
df_compare = df_concat[df_concat['cluster_id'].isin(df_set['cluster_id'].tolist())]
#merge with set to get amount of tables per cluster in overview
df_compare = df_compare.merge(df_set, left_on=['cluster_id'], right_on = ['cluster_id'], how='left')

In [67]:
df_compare

,cluster_id,url,row_id,table_id,name,description,brand,tokens,category,Amount
0,1001446,https://www.bts.com.au/store/p102/Apple_Lightning_Digital_AV_Adapter.html,90,Product_bts.com.au_September2020.json.gz,apple lightning digital av adapter,"Use the Lightning Digital AV Adapter with your iPhone, iPad or iPod with Lightning connector. The Lightning Digital AV Adapter supports mirroring of what is displayed on your device screen — including apps, presentations, websites, slideshows and more — to your HDMI-equipped TV, display, projector or other compatible display in up to 1080p HD.It also outputs video content — movies, TV shows, captured video — to your big screen in up to 1080p HD. Simply attach the Lightning Digital AV Adapter to the Lightning connector on your device and then to your TV or projector via an HDMI cable (sold separately).",NaN,"['apple', 'lightning', 'digital', 'adapter']",electronics,17
1,2859551,https://www.icentre.com.mt/apple-lightning-to-usb-cable-1m-2,511,Product_icentre.com.mt_September2020.json.gz,apple lightning to usb cable,"This USB 2.0 cable connects your iPhone, iPad, or iPod with Lightning connector to your computer’s USB port for syncing and charging. Or you can connect to the Apple USB Power Adapter for convenient charging from a wall outlet.",NaN,"['apple', 'lightning', 'usb', 'cable']",electronics,18
2,323989,https://store.macs-4-u.com.au/usb-c-charge-cable-1-m.html,192,Product_macs-4-u.com.au_September2020.json.gz,usb-c charge cable (1 m),USB-C Charge Cable (1 m),NaN,"['usb-c', 'charge', 'cable', '1']",electronics,11
3,210849,https://www.cingolanibikeshop.com/specialized-sella-power-comp-nero-143.html,447,Product_cingolanibikeshop.com_September2020.json.gz,specialized sella power comp nero 143,SELLA POWER COMP NERO 143,Specialized,"['specialized', 'sella', 'power', 'comp', 'nero', '143']",electronics,2
4,345616,https://www.scan.co.uk/products/6tb-wd-red-pro-wd6003ffbx-35-nas-hdd-sata-iii-6gb-s-7200rpm-256mb-cache-shock-sensor-ncq-oem,4945,Product_scan.co.uk_September2020.json.gz,"wd red pro 6tb 3.5\"" sata nas hdd/hard drive","6TB WD Red Pro WD6003FFBX, 3.5\"" NAS HDD, SATA III 6Gb/s, 7200rpm, 256MB Cache, Shock Sensor, NCQ, OEM",WD,"['wd', 'red', 'pro', '6tb', '3.5\\', ""''"", 'sata', 'hdd/hard', 'drive']",electronics,9
...,...,...,...,...,...,...,...,...,...,...
15355,434734,https://www.scottscycle.com/product/chamois-buttr-original-formula-chamois-cream-184928-1.htm,69,Product_scottscycle.com_September2020.json.gz,original formula chamois cream,"Chamois Butt’r Original is a non-greasy skin lubricant developed by cyclists for use with all chamois. Chamois Butt’r isn’t just for cyclists! It lubricates, soothes, and softens the skin of all...",NaN,"['original', 'formula', 'chamois', 'cream']",clothes,47
15356,5121322,https://urbanpalms.com/product/lg-bottle-palm-tree-hyophorbe-lagenicaulis,5,Product_urbanpalms.com_September2020.json.gz,lg. bottle palm tree (hyophorbe lagenicaulis),Unique and unlike any other palm Bottle Palm is a distinctive addition to any plant collection. Contained in a planter the Bottle Palm is a shapely centerpiece to any porch patio or well lit indoor area. Native to the Mascarene Islands mature trunks have sometimes attained heights of 20 feet.,Urban Palms,"['lg', 'bottle', 'palm', 'tree', 'hyophorbe', 'lagenicaulis']",clothes,3292
15357,5121322,https://spectrumtelescope.com/product/glass-solar-filter-st1150g,30,Product_spectrumtelescope.com_September2020.json.gz,glass solar filter st1150g,"Your O.D. Measurement:11 1/8” (283mm)to11 3/8” (289mm) Fits:Celestron 10\"" Newt;Orion SkyWatcher 10\""",Spectrum Telescope,"['glass', 'solar', 'filter', 'st1150g']",clothes,3292
15358,1512681,https://www.abiza.co.uk/product/raw-garnet-necklace-january-birthday-gifts-raw-stone-necklace-garnet-jewelry,32,Product_abiza.co.uk_September2020.json.gz,raw garnet necklace,Gorgeous deep red Raw Garnet stones in their most natural state have been hand crafted into pendants and suspending fr

In [68]:
#use tokenizer for product names to get tokes for training the model
df_compare['product_tokes'] = df_compare['name'].apply(lambda row: word_tokenize(row))
df_compare['product_tokes'] = remove_stopwords(df_compare['product_tokes'],stopwords.words())
df_compare['product_tokes'] = remove_punctuation (df_compare['product_tokes'])
#get tagged words
tagged_data = [TaggedDocument(words=_d, tags=[str(i)]) for i, _d in enumerate(df_compare['product_tokes'])]
# build model and vocabulary
model = Doc2Vec(vector_size=50, min_count = 5, epochs = 25, dm = 0)
model.build_vocab(tagged_data)
# Train model
model.train(tagged_data, total_examples=model.corpus_count, epochs=25)

In [69]:
#get cluster ids for basline products and with that indices of top products to use model
#1524820,47566,6076,14418,28307,33570,39040,51314,99153,215254,685416, 984421 , 1808651,2887810,34506065,47841827,620473,56116,94055, 150211,182246, 516888, 562955 
top_clusters_list = df_10['cluster_id'].tolist()
index_top_clusters_list=[]
for id in top_clusters_list:
    index_top_clusters_list.append(df_compare[df_compare['cluster_id']==id].index[0])

In [71]:
# get most similar products for each of the base clusters and save them if they have more than 5 tables
clusters_search=[]
for i in index_top_clusters_list:
    similar_doc = model.docvecs.most_similar(f'{i}', topn = 20)
    clusters_search.append(int(i))
    for index, similarity in similar_doc:
        if df_compare.iloc[int(index)]['Amount']>8:
            clusters_search.append(int(index))
    jaccard_score = df_compare['product_tokes'].apply(lambda row: jaccard_similarity_score(row,df_compare.iloc[int(i)]['product_tokes']) )
    indizes=sorted(range(len(jaccard_score)), key=lambda i: jaccard_score[i])[-20:]
    for index in indizes:
         if df_compare.iloc[int(index)]['Amount']>8:
            clusters_search.append(int(index))    
df_final = df_compare.iloc[clusters_search]

In [72]:
df_final.drop_duplicates('cluster_id', keep='first').to_excel("Final_v1.xlsx")

In [ ]:
df_joined_electronics = pd.read_json(os.path.join(mapping_corpus_path_2, 'joined_electronics_v3.json'), compression='gzip', orient='records', lines=True)
#df_joined_clothes = pd.read_json(os.path.join(mapping_corpus_path_2, 'joined_clothes_v3.json'), compression='gzip', orient='records', lines=True)
df_joined_bikes = pd.read_json(os.path.join(mapping_corpus_path_2, 'joined_bikes.json'), compression='gzip', orient='records', lines=True)
df_joined_cars = pd.read_json(os.path.join(mapping_corpus_path_2, 'joined_cars.json'), compression='gzip', orient='records', lines=True)
df_joined_drugstore = pd.read_json(os.path.join(mapping_corpus_path_2, 'joined_drugstore.json'), compression='gzip', orient='records', lines=True)
df_joined_technology = pd.read_json(os.path.join(mapping_corpus_path_2, 'joined_technology.json'), compression='gzip', orient='records', lines=True)
df_joined_tools = pd.read_json(os.path.join(mapping_corpus_path_2, 'joined_tools.json'), compression='gzip', orient='records', lines=True)


In [ ]:
df_grouped_electronics = df_joined_electronics.groupby('cluster_id').count()
#df_grouped_clothes = df_joined_clothes.groupby('cluster_id').count()
df_grouped_bikes = df_joined_bikes.groupby('cluster_id').count()
df_grouped_cars = df_joined_cars.groupby('cluster_id').count()
df_grouped_drugstore = df_joined_drugstore.groupby('cluster_id').count()
df_grouped_technology = df_joined_technology.groupby('cluster_id').count()
df_grouped_tools = df_joined_tools.groupby('cluster_id').count()

In [ ]:
# only look at clusters that have at least one brand associated
df_set_electronics = df_grouped_electronics[df_grouped_electronics['brand_y']>0].reset_index()[['cluster_id','table_id']].rename(columns={'table_id':'Amount'})
#df_set_clothes = df_grouped_clothes[df_grouped_clothes['brand_y']>0].reset_index()[['cluster_id','table_id']].rename(columns={'table_id':'Amount'})
df_set_bikes = df_grouped_bikes[df_grouped_bikes['brand_y']>0].reset_index()[['cluster_id','table_id']].rename(columns={'table_id':'Amount'})
df_set_cars = df_grouped_cars[df_grouped_cars['brand_y']>0].reset_index()[['cluster_id','table_id']].rename(columns={'table_id':'Amount'})
df_set_drugstore = df_grouped_drugstore[df_grouped_drugstore['brand_y']>0].reset_index()[['cluster_id','table_id']].rename(columns={'table_id':'Amount'})
df_set_techonlogy = df_grouped_technology[df_grouped_technology['brand_y']>0].reset_index()[['cluster_id','table_id']].rename(columns={'table_id':'Amount'})
df_set_tools = df_grouped_tools[df_grouped_tools['brand_y']>0].reset_index()[['cluster_id','table_id']].rename(columns={'table_id':'Amount'})

In [ ]:
# We discard all clusters with less than 2 entries, cause we cannot match anything there, so 1,6 million clusters remain
df_set_electronics[df_set_electronics['Amount']>8].to_csv(os.path.join(mapping_corpus_path_2, 'Electronics_cluster_8_tables.csv'))
#df_set_clothes[df_set_clothes['Amount']>8].to_csv(os.path.join(mapping_corpus_path_2, 'Clothes_cluster_8_tables.csv'))
df_set_bikes[df_set_bikes['Amount']>8].to_csv(os.path.join(mapping_corpus_path_2, 'Bikes_cluster_8_tables.csv'))
df_set_cars[df_set_cars['Amount']>8].to_csv(os.path.join(mapping_corpus_path_2, 'Cars_cluster_8_tables.csv'))
df_set_drugstore[df_set_drugstore['Amount']>8].to_csv(os.path.join(mapping_corpus_path_2, 'Drugstore_cluster_8_tables.csv'))
df_set_techonlogy[df_set_techonlogy['Amount']>8].to_csv(os.path.join(mapping_corpus_path_2, 'Technology_cluster_8_tables.csv'))
df_set_tools[df_set_tools['Amount']>8].to.to_csv(os.path.join(mapping_corpus_path_2, 'Tools_cluster_8_tables.csv'))